In [2]:
# download each subject from google drive
! gdown --id 1dd7-hzTbhffR2mqRPok9wbtAbWWfoUqW
! unzip economics.zip

! gdown --id 1YRH6TL6uPTSweszOOcPUT2tvzVbI_zJS
! unzip politics.zip

! gdown --id 1L0OoEOy2XJUgIDbnoWh-l8bhK31kOE9I
! unzip sports.zip

! gdown --id 1LskvRA05fTXcTMgWsvfd302C8nG2Eq_o
! unzip cultural.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1dd7-hzTbhffR2mqRPok9wbtAbWWfoUqW
To: /content/economics.zip
100% 147M/147M [00:00<00:00, 286MB/s]
Archive:  economics.zip
replace economics.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1YRH6TL6uPTSweszOOcPUT2tvzVbI_zJS
To: /content/politics.zip
100% 101M/101M [00:00<00:00, 294MB/s] 
Archive:  politics.zip
replace politics.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option

In [3]:
!git clone https://github.com/language-ml/2-nlp-language-modeling.git

Cloning into '2-nlp-language-modeling'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 159 (delta 108), reused 133 (delta 106), pack-reused 23
Receiving objects: 100% (159/159), 44.31 MiB | 18.08 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [4]:
!mv 2-nlp-language-modeling/* ./
!rm -rf 2-nlp-language-modeling

In [5]:
!pip install hazm
!pip install transformers
!pip install strsimpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 12.5 MB/s 
     |████████████████████████████████| 233 kB 71.1 MB/s 
     |████████████████████████████████| 1.4 MB 63.8 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=fa5154ee3fb477b9de6ee483408d77e172a0cbc41da9592c87c5aa406ff707c3
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154425 sha256=010aa3e805692fdd4086b63f79e70ccf65d187319dd40846c199957ec8e99529
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
Looking in indexes: h

In [6]:
import random
import time
import numpy as np
import nltk
import pandas as pd
import numpy as np
import tqdm
import codecs
from itertools import product
import math
import itertools
import random
from collections import Counter, defaultdict
from pprint import pprint
from __future__ import unicode_literals
from hazm import *
from strsimpy.weighted_levenshtein import WeightedLevenshtein
import torch 
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoConfig, AutoTokenizer, BertForMaskedLM

In [7]:
def read_file_and_sample(file, sample_size):
  file_lines = []
  with open(file) as f:
    for line in f:
      file_lines.append(line)
    sampled_file = random.sample(file_lines, sample_size)
  return sampled_file

In [8]:
sampled_politics = read_file_and_sample('politics.txt', 300000)
sampled_economics = read_file_and_sample('economics.txt',300000)
sampled_sports = read_file_and_sample('sports.txt',300000)
sampled_cultural = read_file_and_sample('cultural.txt',300000)
# combining all samples
all_samples = sampled_politics + sampled_economics + sampled_sports + sampled_cultural

In [9]:
normalizer = Normalizer()
all_samples_normalized = [normalizer.normalize(x) for x in tqdm.tqdm_notebook(all_samples)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1200000 [00:00<?, ?it/s]

In [10]:
all_sentences = [sent_tokenize(x) for x in tqdm.tqdm_notebook(all_samples_normalized)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/1200000 [00:00<?, ?it/s]

In [11]:
sentences = list(itertools.chain(*all_sentences))

In [12]:
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('farsi/stopwords.txt','r','utf-8').readlines()]

In [13]:
class LanguageModel(object):
    """An n-gram language model trained on a given corpus.
    
    For a given n and given training corpus, constructs an n-gram language
    model for the corpus by:
    1. preprocessing the corpus (adding SOS/EOS/UNK tokens)
    2. calculating (smoothed) probabilities for each n-gram
    Also contains methods for calculating the perplexity of the model
    against another corpus, and for generating sentences.
    Args:
        train_data (list of str): list of sentences comprising the training corpus.
        n (int): the order of language model to build (i.e. 1 for unigram, 2 for bigram, etc.).
        laplace (int): lambda multiplier to use for laplace smoothing (default 1 for add-1 smoothing).
    """

    SOS = "<s>"
    EOS = "</s>"
    UNK = "<UNK>"
    
    def __init__(self, train_data, n, laplace=1):
        self.n = n
        self.vocab = dict()
        self.laplace = laplace
        self.tokens = self.preprocess(train_data, n)
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):
        """Apply Laplace smoothing to n-gram frequency distribution.
        
        Here, n_grams refers to the n-grams of the tokens in the training corpus,
        while m_grams refers to the first (n-1) tokens of each n-gram.
        Returns:
            dict: Mapping of each n-gram (tuple of str) to its Laplace-smoothed 
            probability (float).
        """
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):
        """Create a probability distribution for the vocabulary of the training corpus.
        
        If building a unigram model, the probabilities are simple relative frequencies
        of each token with the entire corpus.
        Otherwise, the probabilities are Laplace-smoothed relative frequencies.
        Returns:
            A dict mapping each n-gram (tuple of str) to its probability (float).
        """
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        """Convert, if necessary, a given n-gram to one which is known by the model.
        Starting with the unmodified ngram, check each possible permutation of the n-gram
        with each index of the n-gram containing either the original token or <UNK>. Stop
        when the model contains an entry for that permutation.
        This is achieved by creating a 'bitmask' for the n-gram tuple, and swapping out
        each flagged token for <UNK>. Thus, in the worst case, this function checks 2^n
        possible n-grams before returning.
        Returns:
            The n-gram with <UNK> tokens in certain positions such that the model
            contains an entry for it.
        """
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def sent_probability(self, test_data):
        test_tokens = test_data.split()
        test_tokens = self.replace_singletons(test_tokens)
        known_ngram = self._convert_oov(tuple(test_tokens))
        prob = self.model[known_ngram]
        return prob

    def perplexity(self, test_data):
        """Calculate the perplexity of the model against a given test corpus.
        
        Args:
            test_data (list of str): sentences comprising the training corpus.
        Returns:
            The perplexity of the model as a float.
        
        """
        test_tokens = self.preprocess(test_data, self.n)
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        N = len(test_tokens)

        known_ngrams  = [self._convert_oov(ngram) for ngram in test_ngrams]
        probabilities = [self.model[ngram] for ngram in known_ngrams]
        
        for x,y in zip(known_ngrams, probabilities):
            print(x,y)
        
        return math.exp((-1/N) * sum(map(math.log, probabilities)))

    def _best_candidate(self, prev, without=[]):
        
        blacklist  = [LanguageModel.UNK] + without

        if len(prev) < self.n:
            prev = [LanguageModel.SOS]*(self.n-1)

        candidates = list(((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==tuple(prev)))

        probs = [y for x,y in candidates]
        probs = probs/np.sum(probs)
        words = [x for x,y in candidates]

        idx = np.random.choice(len(words), 1, replace=False, p=probs)[0]
        
        while words[idx] in blacklist:
            idx = np.random.choice(len(words), 1, replace=False, p=probs)[0]
        
        return (words[idx], probs[idx])
         
    def generate_sentence(self, min_len=12, max_len=24):
        sent, prob = ([LanguageModel.SOS] * (max(1, self.n-1)), 1)
        while sent[-1] != LanguageModel.EOS:
            prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
            blacklist = sent + ([LanguageModel.EOS,LanguageModel.SOS] if len(sent) < min_len else [])
            next_token, next_prob = self._best_candidate(prev, without=blacklist)
            sent.append(next_token)
            prob *= next_prob

            if len(sent) >= max_len:
                sent.append(LanguageModel.EOS)

        return (' '.join(sent[(self.n-1):-1]), -1/math.log(prob))
    
    

    def add_sentence_tokens(self, sentences, n):
        """Wrap each sentence in SOS and EOS tokens.
        For n >= 2, n-1 SOS tokens are added, otherwise only one is added.
        Args:
            sentences (list of str): the sentences to wrap.
            n (int): order of the n-gram model which will use these sentences.
        Returns:
            List of sentences with SOS and EOS tokens wrapped around them.
        """
        sos = ' '.join([LanguageModel.SOS] * (n-1)) if n > 1 else LanguageModel.SOS
        return ['{} {} {}'.format(sos, s, LanguageModel.EOS) for s in sentences]

    def replace_singletons(self, tokens):
        """Replace tokens which appear only once in the corpus with <UNK>.

        Args:
            tokens (list of str): the tokens comprising the corpus.
        Returns:
            The same list of tokens with each singleton replaced by <UNK>.

        """
        if len(self.vocab) == 0:
            self.vocab = nltk.FreqDist(tokens)
        return [token if self.vocab[token] > 1 else LanguageModel.UNK for token in tokens]

    def preprocess(self, sentences, n):
        """Add SOS/EOS/UNK tokens to given sentences and tokenize.
        Args:
            sentences (list of str): the sentences to preprocess.
            n (int): order of the n-gram model which will use these sentences.
        Returns:
            The preprocessed sentences, tokenized by words.
        """
        sentences = self.add_sentence_tokens(sentences, n)
        tokens = ' '.join(sentences).split()
        tokens = self.replace_singletons(tokens)
        return tokens    

In [14]:
#3_gram module
language_model = LanguageModel(sentences, 3)

In [32]:
#load model v3.0
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
vocabulary = [tokenizer.decode(idx) for idx in range(42000)]
for j, voc in enumerate(vocabulary):
    if "##" in voc:
        vocabulary[j] = voc[2:]
model = BertForMaskedLM.from_pretrained(model_name_or_path)

In [33]:
def insertion_cost(char):
    return 1.5

def deletion_cost(char):
    return 1.5

def substitution_cost(char_a, char_b):
    return 1.0

weighted_levenshtein = WeightedLevenshtein(
    substitution_cost_fn=substitution_cost,
    insertion_cost_fn=insertion_cost,
    deletion_cost_fn=deletion_cost)



In [34]:
scaler = MinMaxScaler(feature_range=(0, 10))

In [35]:
def combine_scores(parsbert_scores, meds, n_gram_scores, alpha=1, beta=5, gamma=1):
    return alpha*parsbert_scores - beta*meds + gamma*n_gram_scores

In [36]:
def farsi_spell_correction(text, token, idx):
  inputs = tokenizer(text, return_tensors='pt')
  with torch.no_grad():
    logits = model(**inputs).logits.squeeze()
  mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
  predicted_token_id = logits[mask_token_index]
  copy_parsbert_scores = torch.clone(predicted_token_id)
  for _ in range(500):
      i = copy_parsbert_scores.argmax()
      if weighted_levenshtein.distance(tokenizer.decode(i), token) < 2:
          return tokenizer.decode(i)
      copy_parsbert_scores[0, i] = -np.Inf
  del copy_parsbert_scores    
  predicted_token_id.resize_(42000, 1)
  scaled_parsbert_scores = scaler.fit_transform(predicted_token_id)
  meds = np.array([weighted_levenshtein.distance(token, vocab) for vocab in vocabulary])
  scaled_meds = scaler.fit_transform(meds.reshape(-1, 1))
  text = '<s> ' + text + ' </s>'
  text = text.split()
  n_gram_scores = [language_model.sent_probability(text[idx-1]+" "+vocab+ " "+text[idx+1]) for vocab in vocabulary][0]
  n_gram_scores = np.array(n_gram_scores)
  n_gram_socres = scaler.fit_transform(n_gram_scores.reshape(-1, 1))
  combined_scores = combine_scores(scaled_parsbert_scores, scaled_meds, n_gram_scores, 1, 5, 1)
  return tokenizer.decode(combined_scores.argmax())

In [37]:
while True:
  sent = input("لطفا جمله‌ای بنویسید: ")
  time_start = time.time()
  sent = sent.split()
  for idx in range(len(sent)):
    token = sent[idx]
    sent[idx] = '[MASK]'
    res = farsi_spell_correction(" ".join(sent), token, idx)
    sent[idx] = res
  result = " ".join(sent)
  print("پس از تصحیح : ", result)
  print(f"It took: {time.time() - time_start:.4f}s")
  print("-"*50)
  exit = input("(بله)می‌خواهید خارج شوید؟ ")
  if exit == 'بله':
      break

لطفا جمله‌ای بنویسید: ظعفران غضا را بسیار خوشمذه می‌کند
پس از تصحیح :  زعفران غذا را بسیار خوشمزه میکند
It took: 2.4351s
--------------------------------------------------
(بله)می‌خواهید خارج شوید؟ 
لطفا جمله‌ای بنویسید: شهنامه فردوسی بهترین اسر ایزان است
پس از تصحیح :  شاهنامه فردوسی بهترین اثر ایران است
It took: 2.2757s
--------------------------------------------------
(بله)می‌خواهید خارج شوید؟ 
لطفا جمله‌ای بنویسید: وقتی آسمان پرستاره را با ائجاب مرور می کنیم یا بر ساهل دریا یا رودخـانه می نشینیم وقت خود را تلف نکرده ایم این ها جزع زندگی است به خصوص اگر با تأمّل و فکر و ابرت همراه باشد
پس از تصحیح :  وقتی اسمان پرستاره را بر اعجاب مرور مى کنیم یا در ساحل دریا یا رودخانه می نشینیم وقتی خود را تلف کرده ایم این هم جز زندگی است به خصوص اگر با تامل و تفکر و عبرت همراه باشید
It took: 9.6510s
--------------------------------------------------
(بله)می‌خواهید خارج شوید؟ 
لطفا جمله‌ای بنویسید: امروز در اخرین لحضات امتحان حالم بد شد
پس از تصحیح :  امروز در اخرین لحظات امتحان حالم بد شد
It too

In [20]:
'''
ظعفران غضا را بسیار خوشمذه می‌کند
شهنامه فردوسی بهترین اسر ایزان است
وقتی آسمان پرستاره را با ائجاب مرور می کنیم یا بر ساهل دریا یا رودخـانه می نشینیم وقت خود را تلف نکرده ایم این ها جزع زندگی است به خصوص اگر با تأمّل و فکر و ابرت همراه باشد
امروز در اخرین لحضات امتحان حالم بد شد
همد صصد و نهمین صوره از سوره‌های فرآن است
به گزارش خبرنگار انتذامی خبرگزاری فارس، صرقت از صندوق امانات بانک طجارت شعبه دانشگاه خبری بود که روز گذشته منتشر شد
تأیید منشأ ارز مورد نیاز برای واردات خودرو از سوی بانک مرکضی، مهمترین الذام پیش روی واردکنندگان خودرو خواهد بود
متأثفانه در ساعت ۵:۳۰ دقیقه بامداد چهارشنبه هجدهم خرداد ماه غظار مصافربری در کیلومتر ۵۰ طبس به یزد از ریل خارج شد
ساخت موشک‌های بالستیک ایران به طور خاس طهدیدی برای (رژیم موقت) اسرائیل و نظامیان ایالات مطحده در خاورمیانه است
دبیر انجمن شرکت های حمل و نقل ریلی: پیش از این حشدار‌های زیادی در رابطه با وقوع صوانح در حوزه حمل و نقل ریلی داده بودیم
'''